In [1]:
import bempp.api
import numpy as np

import sys
sys.path.append('../../')
from decimal import *

import bemppUQ
from bemppUQ.operators.maxwell import assemble_operators, evaluate_far_field, evaluate_far_field_sd

bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'
bempp.api.global_parameters.hmat.eps = 1E-4


from bemppUQ.shapes import reentrant_cube_exterior

def function(x, y, i, j):
    zx = 0
    zy = 0
    if i == 0:
        zx = np.sin(x * np.pi * 2)
    if i == 1:
        if x <= 0.25:
            zx = np.sin(x * np.pi * 4)
    if i == 2:
        if x > 0.25:
            zx = -np.sin(x * np.pi * 4)
    if i == 3:
        if x <= 0.5 / 3.:
            zx = np.sin(x * np.pi * 6)
    if i == 4:
        if x > 0.5 / 3. and x <= 1 / 3.:
            zx = -np.sin(x * np.pi * 6)
    if i ==5:
        if x > 1/3.:
            zx = np.sin(x * np.pi * 6)

    if j == 0:
        zy = np.sin(y * np.pi * 2)

    if j == 1:
        if y <= 0.25:
            zy = np.sin(y * np.pi * 4)

    if j == 2:
        if y > 0.25:
            zy = -np.sin(y * np.pi * 4)
    
    if j == 3:
        if y <= 0.5 / 3.:
            zy = np.sin(y * np.pi * 6)

    if j == 4:
        if y > 0.5 / 3. and y <= 1 / 3.:
            zy = -np.sin(y * np.pi * 6)
    if j == 5:
        if y > 1/3.:
            zy = np.sin(y * np.pi * 6)

    return zx * zy


# Def random:

def Phiz(point, n, domain_index, result):
    x, y, z = point

    res = 0j
    if z == 0.5 and (x <= 0.5) and (y <= 0.5):
        for ii in range(6):
            for jj in range(6):
                res += Random[ii,jj] * function(x, y, ii,jj)	
    result[0] = res



def perturbate(grid, t, kappa_pert=None):
    P1 = bempp.api.function_space(grid, 'B-P', 1)
    grid_funz = bempp.api.GridFunction(P1, fun = Phiz)
    elements = grid.leaf_view.elements
    vertices = grid.leaf_view.vertices
    normals = P1.global_dof_normals
    x, y, z = vertices
    
    vertices[2, :] = z + t * grid_funz.coefficients
    return bempp.api.grid_from_element_data(vertices, elements)



def get_base_grid(h):

    gr0 = bempp.api.shapes.reentrant_cube(h=h, refinement_factor=1) 
    elements0 = list(gr0.leaf_view.entity_iterator(0))
    N0 = len(elements0)
    tol = h / 10.
    for i in range(N0):
        el0 = elements0[i]
        z = el0.geometry.corners[2]
        if np.linalg.norm(np.array([1/2, 1/2, 1/2]) - z) < tol:
            gr0.mark(el0)
    gr1 = gr0.refine()


    elements0 = list(gr1.leaf_view.entity_iterator(0))
    N0 = len(elements0)
    for i in range(N0):
        el0 = elements0[i]
        z = el0.geometry.corners[2]
        if np.linalg.norm(np.array([1/2, 1/2, 1/2])-z) < tol:
            gr1.mark(el0)

    base_grid = gr1.refine()
    return base_grid


config = bemppUQ.config.set_case('A')

axis = 0
#### define l0 and l1

l0 = 2
l1 = 2

precision_list = [2, 5, 10]

precision0 = precision_list[l0]
precision1 = precision_list[l1]

h0 = 2.0 * np.pi / (precision0 * config["k_int"])
h1 = 2.0 * np.pi / (precision1 * config["k_int"])

# generate the grids

if l0 == 2:
    grid0 = get_base_grid(h=h0) 
else:
    grid0 = bempp.api.shapes.reentrant_cube(h=h0, refinement_factor=1) 

if l1 == 2:
    grid1 = get_base_grid(h=h1)
else:
    grid1 = bempp.api.shapes.reentrant_cube(h=h1, refinement_factor=1) 
    
N0 = 2 * grid0.leaf_view.entity_count(1)
N1 = 2 * grid1.leaf_view.entity_count(1)

print(N0, N1, N0 * N1)

solve = False

3204 3204 10265616


In [2]:
transmission_operators0 = assemble_operators(grid0, config)
transmission_operators1 = assemble_operators(grid1, config)


Umean0, solution0 = evaluate_far_field(transmission_operators0, config)
Umean1, solution1 = evaluate_far_field(transmission_operators1, config)

nrank = 1
grid_funs0 = []
grid_funs1 = []

for ii in range(nrank):
    for jj in range(nrank):
        print(ii,jj)
        def fun(point, n, domain_index, result):
            x, y, z = point

            res = 0j
            if z == 0.5 and (x <= 0.5) and (y <= 0.5):
                res += function(x, y, ii,jj)
            result[0] = res
            
        space0 = bempp.api.function_space(grid0, 'B-P', 1)
        grid_fun0 = bempp.api.GridFunction(space0, fun= fun)
        grid_funs0.append(grid_fun0)
        
        space1 = bempp.api.function_space(grid1, 'B-P', 1)
        grid_fun1 = bempp.api.GridFunction(space1, fun= fun)
        grid_funs1.append(grid_fun1)

iteration - 1 || residual - 0.145570369404 0.7042975425720215
iteration - 2 || residual - 0.0303848264104 0.4438810348510742
iteration - 3 || residual - 0.00667101666613 0.3908212184906006
iteration - 4 || residual - 0.00108642514403 0.3973221778869629
iteration - 5 || residual - 0.000218854097476 0.38612914085388184
iteration - 6 || residual - 4.56938052444e-05 0.3803868293762207
iteration - 7 || residual - 9.49719737069e-06 0.0018696784973144531
iteration - 1 || residual - 0.145570369404 0.7838857173919678
iteration - 2 || residual - 0.0303848264104 0.40669751167297363
iteration - 3 || residual - 0.00667101666613 0.3961811065673828
iteration - 4 || residual - 0.00108642514403 0.42227673530578613
iteration - 5 || residual - 0.000218854097476 0.38553357124328613
iteration - 6 || residual - 4.56938052444e-05 0.44983792304992676
iteration - 7 || residual - 9.49719737069e-06 0.0023543834686279297
0 0


In [3]:
grid_funs0

In [4]:
result0 = evaluate_far_field_sd(grid0, transmission_operators0, config, solution0, grid_funs0, solve = solve)
result1 = evaluate_far_field_sd(grid1, transmission_operators1, config, solution1, grid_funs1, solve = solve)

0
0


In [5]:
result0, lhs_op0 = result0
result1, lhs_op1 = result1

In [6]:
from bemppUQ.assembly.operators import HDenseTensorLinearOperator
from bempp.api.assembly.blocked_operator import \
        coefficients_of_grid_function_list, \
        projections_of_grid_function_list, \
        grid_function_list_from_coefficients

In [7]:
N0 = 2 * grid0.leaf_view.entity_count(1)
N1 = 2 * grid1.leaf_view.entity_count(1)

print(N0)
print(N1)

3204
3204


In [ ]:
n = config['number_of_angles']

C = np.zeros((N0, N1), dtype=np.complex128)
Sigma = np.zeros((n, n), dtype=np.complex128)
k = 0
for ii in range(nrank):
    for jj in range(nrank):
        cc0 = coefficients_of_grid_function_list(result0[k])
        cc1 = coefficients_of_grid_function_list(result1[k])
        C += 1/3. * bemppUQ.utils.login.tensorize(cc0,cc1)
        k += 1
        
t = 0.05

A0 = (lhs_op0 * lhs_op0).strong_form()
A1 = (lhs_op1 * lhs_op1).strong_form()

AA = HDenseTensorLinearOperator(A0, A1)
from bemppUQ.utils.login import gmres
print('0')
test = A0 * C
print('1')
test2 = (A1 * test.conj().T).conj().T
print("2")
import time
ta = time.time()
X, info, res, times = gmres(AA, np.ravel(test2), return_residuals=True)
ta = time.time()-ta
print(ta, "texec")

0


In [ ]:
electric_far0, magnetic_far0 = transmission_operators0[3], transmission_operators0[4] 
electric_far1, magnetic_far1 = transmission_operators1[3], transmission_operators1[4] 

In [ ]:
E0 = bempp.api.as_matrix(electric_far0.discrete_operator)
M0 = bempp.api.as_matrix(magnetic_far0.discrete_operator)
Z0 = np.zeros(E0.shape, dtype = E0.dtype)

E1 = bempp.api.as_matrix(electric_far1.discrete_operator)
M1 = bempp.api.as_matrix(magnetic_far1.discrete_operator)
Z1 = np.zeros(E1.shape, dtype = E1.dtype)

R0 = np.bmat([Z0, -E0], [-M0, Z0])
R1 = np.bmat([Z1, -E1], [-M1, Z1])

In [ ]:
R0.shape

In [ ]:
X = X.reshape(C.shape)

In [ ]:
X.shape

In [ ]:
R0.shape

In [ ]:
Xtemp = np.dot(R0, X)
FF = np.dot(Xtemp, R1.T)

In [ ]:
Uvar = t ** 2 * FF.diagonal()

In [ ]:
Uvars = np.array(np.sum(Uvar.reshape(3,400), axis=0)).ravel()
Umeans = np.sum(Umean0, axis=0)

In [ ]:
from matplotlib import pyplot as plt

angles = config['angles']

fig, (ax1, ax2)  = plt.subplots(1,2, figsize=(12,6))

cs = 10 * np.log10(4 * np.pi * np.abs(Umeans)**2)

ax1.plot(angles, cs)


y = np.sqrt(np.abs(Uvars))
ax2.plot(angles, y)


ax2.set_xlabel(r'$\theta$ (rad)', fontsize=14)
ax2.set_ylabel(r'$\mathrm{RCS}_z(\theta)$', fontsize=14)


ax1.set_xlabel(r'$\theta$ (rad)', fontsize=14)
ax1.set_ylabel(r'$\mathrm{RCS}(\theta)$', fontsize=14)

ax1.legend()
ax1.grid()

ax2.legend()
ax2.grid()
ax2.set_ylim(0, 0.013)

plt.show(block=False)
